## Imports

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as T
import bcolz
import time
from torch.utils.data import Dataset, DataLoader, sampler

%matplotlib inline

In [2]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

Using gpu: True 


## Data processing

In [3]:
data_dir = './Dataset'

transform = T.Compose([
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class ImageFolderWithPaths(dset.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

train_dataset = ImageFolderWithPaths('./Dataset/train', transform=transform)
val_dataset = ImageFolderWithPaths('./Dataset/val', transform=transform)
test_dataset = ImageFolderWithPaths('./Dataset/test', transform=transform)

In [4]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)
print("Number of training examples {}, validation examples {}, testing examples {}".format(train_size, val_size, test_size))

Number of training examples 66071, validation examples 11016, testing examples 33154


In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=6)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
c = 0
for inputs, labels, paths in train_dataloader:
    # use the above variables freely
    print(inputs.shape, labels.shape, paths)
    c += 1
    if c == 1:
        break

torch.Size([64, 3, 224, 224]) torch.Size([64]) ('./Dataset/train/54/15_42.jpg', './Dataset/train/42/6_22.jpg', './Dataset/train/101/11_39.jpg', './Dataset/train/161/9_38.jpg', './Dataset/train/62/13_52.jpg', './Dataset/train/22/7_5.jpg', './Dataset/train/55/20_29.jpg', './Dataset/train/109/4_11.jpg', './Dataset/train/15/19_26.jpg', './Dataset/train/82/8_32.jpg', './Dataset/train/53/7_6.jpg', './Dataset/train/103/1_36.jpg', './Dataset/train/148/13_11.jpg', './Dataset/train/16/21_18.jpg', './Dataset/train/105/19_20.jpg', './Dataset/train/67/15_31.jpg', './Dataset/train/71/2_22.jpg', './Dataset/train/86/1_20.jpg', './Dataset/train/97/8_0.jpg', './Dataset/train/13/13_43.jpg', './Dataset/train/106/afe37626ae96661e2658b69f23548be0.jpg', './Dataset/train/109/13_35.jpg', './Dataset/train/97/19_7.jpg', './Dataset/train/88/2_33.jpg', './Dataset/train/3/1_22.jpg', './Dataset/train/37/14_36.jpg', './Dataset/train/93/20_17.jpg', './Dataset/train/143/12_12.jpg', './Dataset/train/164/13_2.jpg', './Da

In [6]:
IngreLabel = open('IngreLabel.txt', 'r').read().split('\n')  # list of str
for i in range(len(IngreLabel)):
    IngreLabel[i] = IngreLabel[i].split()                    # list of list, element is str


In [6]:
IngreLabel = open('IngreLabel.txt', 'r').read().split('\n')[:-1]  # list of str
path_to_ingredients = dict()
for i in range(len(IngreLabel)):
    path_and_ingredients = IngreLabel[i].split()
    path, ingredients = path_and_ingredients[0], [0 if int(label) == -1 else int(label) for label in path_and_ingredients[1:]]
    path_to_ingredients[path] = np.array(ingredients)

In [1]:
print(path_to_ingredients['/100/xiachufang_1.jpg'].shape)
print(len(path_to_ingredients['/100/xiachufang_1.jpg']))
print(len(path_to_ingredients))

NameError: name 'path_to_ingredients' is not defined

In [8]:
def get_ingredients(path):
    split_path = path.split("/")
    key = '/' + '/'.join(split_path[3:])
    ingredients = path_to_ingredients[key]
    return ingredients

## Creating VGG model

In [9]:
model_vgg = models.vgg16(pretrained=True)
print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
model_vgg = gpu(model_vgg)

### Calculating preconvoluted features

In [11]:
def preconvfeat_val(dataset):
    ''' Calculates the outputs of model_vgg.features (i.e., before the FC layers) on the validation dataset.
        Returns a list of the features generated for each image, and a list of their corresponding labels.'''
    conv_features = []
    food_labels_list = []
    ingredient_labels_list = []
    i = 0 # Counter for how many batches we have processed so far.
    for data in dataset:
        if i % 10 == 0:
            print("Completed {} forwarded passes".format(i))
        inputs, food_labels, paths = data
        ingredient_labels = []
        for path in paths:
            ingredient_labels.append(get_ingredients(path))
        inputs = gpu(inputs)
        food_labels = gpu(food_labels)
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        food_labels_list.extend(food_labels.data.cpu().numpy())
        ingredient_labels_list.extend(ingredient_labels)
        i += 1
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features, food_labels_list, ingredient_labels_list)

In [12]:
def preconvfeat_train(dataset, batches_per_file=100):
    ''' Calculates the outputs of model_vgg.features (i.e., before the FC layers) on the training dataset.
        Returns a list of the features generated for each image, and a list of their corresponding labels.'''
    output_conv_features = [] # Contains a list of lists features. Each sub-list will be saved in its own file.
    conv_features = []
    output_food_labels_list = []
    food_labels_list = []
    output_ingredient_labels_list = []
    ingredient_labels_list = []
    i = 0 # Counter for how many batches we have processed so far.
    for data in dataset:
        # Once we have made 'batches_per_file' forward passes, we concatenate the features for each of those 
        # forward passes together, and add them to output_conv_features. This is done mainly to limit the size of 
        # the number of features we're concatenating together for the large training dataset.
        if i >= batches_per_file:
            print("Completed {} forward passes".format(batches_per_file))
            i = 0
            output_conv_features.append(np.concatenate([[feat] for feat in conv_features]))
            output_food_labels_list.append(food_labels_list)
            output_ingredient_labels_list.append(ingredient_labels_list)
            conv_features = []
            food_labels_list = []
            ingredient_labels_list = []
        inputs, food_labels, paths = data
        ingredient_labels = []
        for path in paths:
            ingredient_labels.append(get_ingredients(path))
        inputs = gpu(inputs)
        labels = gpu(food_labels)
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        food_labels_list.extend(labels.data.cpu().numpy())
        ingredient_labels_list.extend(ingredient_labels)
        i += 1
    if len(conv_features) > 0:
        output_conv_features.append(np.concatenate([[feat] for feat in conv_features]))
        output_food_labels_list.append(food_labels_list)
        output_ingredient_labels_list.append(ingredient_labels_list)
    return (output_conv_features, output_food_labels_list, output_ingredient_labels_list)

In [13]:
%%time
conv_feat_train, food_labels_train, ingredient_labels_train = preconvfeat_train(train_dataloader)

Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
Completed 100 forward passes
CPU times: user 7min 42s, sys: 2min 28s, total: 10min 10s
Wall time: 10min 17s


In [13]:
%%time
conv_feat_val, food_labels_val, ingredient_labels_val = preconvfeat_val(val_dataloader)

Completed 0 forwarded passes
Completed 10 forwarded passes
Completed 20 forwarded passes
Completed 30 forwarded passes
Completed 40 forwarded passes
Completed 50 forwarded passes
Completed 60 forwarded passes
Completed 70 forwarded passes
Completed 80 forwarded passes
Completed 90 forwarded passes
Completed 100 forwarded passes
Completed 110 forwarded passes
Completed 120 forwarded passes
Completed 130 forwarded passes
Completed 140 forwarded passes
Completed 150 forwarded passes
Completed 160 forwarded passes
Completed 170 forwarded passes
CPU times: user 1min 17s, sys: 25.6 s, total: 1min 42s
Wall time: 1min 45s


In [14]:
def save_array_train(fname, arr):
    for i, arr_list in enumerate(arr): 
        c=bcolz.carray(arr_list, rootdir=fname + "_" + str(i) + '.bc', mode='w')
    c.flush()

def save_array_val(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

%mkdir ./ingredients/vgg16

mkdir: cannot create directory ‘./ingredients/vgg16’: File exists


In [15]:
save_array_train(os.path.join('./ingredients/vgg16','feat_train'), conv_feat_train)
save_array_train(os.path.join('./ingredients/vgg16','food_labels_train'), food_labels_train)
save_array_train(os.path.join('./ingredients/vgg16','ingredient_labels_train'), ingredient_labels_train)

# save_array_val(os.path.join('./ingredients/vgg16','feat_val.bc'), conv_feat_val)
# save_array_val(os.path.join('./ingredients/vgg16','food_labels_val.bc'), food_labels_val)
# save_array_val(os.path.join('./ingredients/vgg16','ingredient_labels_val.bc'), ingredient_labels_val)

## Training fully connected module

### Loading Preconvoluted features

In [27]:
conv_feat_val = load_array('./vgg16/feat_val.bc')
labels_val = load_array('./vgg16/labels_val.bc')

In [28]:
print(len(conv_feat_val))
print(conv_feat_val.shape)

11016
(11016, 512, 7, 7)
